### Initiation

In [1]:
#os.chdir('C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Lab Management/Dry Lab/Python/Python Scripts/LabScripts')
# To manipulate arrays
import numpy as np 
# To import images 

# To handle track DataFrames
import pandas as pd

# To import images
from skimage import io 
from skimage.io import imread

# For TrackPy
import trackpy as tp

# To make plots
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()  

# To work inline; change to %matplotlib notebook for interactive plotting
%matplotlib inline 

# Napari 
%gui qt5 
from skimage import data
import napari

# To create interactive elements
import ipywidgets as widgets 
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout
from ipywidgets.embed import embed_minimal_html, dependency_state

# Image processing and filters
from skimage.filters import difference_of_gaussians

# Iteration tools such as groupby 
import itertools

# For directories 
import os

# For reloading a library when testing
import importlib

# For deleting/reloading modules in another file for code testing
import sys

# For statistics
from scipy import stats

# For curve fitting
from scipy.optimize import curve_fit

# Seaborn plotting
import seaborn as sns

# Import trackArrayTools
from trackArrayTools import *

# Import for correlation analysis
from scipy import signal

#rSNAPsim_IP
#cwd = os.getcwd()  # get current working director
#os.chdir('../rSNAPsim_IP/rSNAPsim_IP/') # assume rSNAPsim_IP is  one directory up
#import rSNAPsim_IP as rss_IP #importing rSNAPsim.
#os.chdir(cwd) # change back to original current directory (the directory this file is in)

#del sys.modules['trackArrayTools']  # Use this if you are coding new class methods and want to reread trackArrayTools.py file
#from trackArrayTools import *

### Set current working directory, track array filenames, and basic parameters

In [2]:
# Enter in the following:
crop_pad = 5    # specifies the size of crops in track arary. If 5, then crops are (2*5 + 1 = 11) x 11
xy_pixel_size = 130   # voxel  xy dimensions in crops
z_pixel_size = 500   # voxel z dimension
working_directory = 'C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Our papers/Ago2Tethering/TimFastTrackAnalysis/'
video_3D_filename_path = 'C:/Users/tim_s/Documents/Python Scripts/LabScripts/TestData/'
#beads_path = 'X:/_FiXie/Charlotte/_90m movies for particle tracking/20180707_TA_translation assay_90m images/Beads/'
video_3D_filename = 'TA02_90m.tif'

In [5]:
# For display, so z and xy are shown properly when viewing track array
z_renorm = z_pixel_size/xy_pixel_size  

#  Track array filenames should be the following:
track_array_filename = 'Tracks_' + video_3D_filename[:-4] + '_crop_pad_' + str(crop_pad) + '.tif' 
track_filename = track_array_filename[:-15] + '.csv'
[track_array_filename, track_filename]

['Tracks_TA02_90m_crop_pad_5.tif', 'Tracks_TA02_90m.csv']

### Creating Track Array (only done once using original 3D video and tracks)

In [ ]:
# # Loading the figure with beads so track array can be registered
# # Dimension 1 is the number of channel. Red and Green. # Dimension 2 is the x axis. # Dimension 3 is the y axis.
# figWithBeads = beads_path + 'Beads01.tif'
# im_beads = io.imread(figWithBeads) # reading the image

In [ ]:
# # Using rSNAPSim to get the homography matrix to align the green/blue channels with red
# #temp_obj_beads = rss_IP.BeadsAlignment(im_beads)
# [my_homography,positions_green,positions_red] = temp_obj_beads.make_beads_alignment()  # !!! Adjusted Luis' code to spit this info out

In [ ]:
# Check that the homography matrix is working:
#my_homography_inverse = np.linalg.inv(my_homography)  # This is the transformation we use to shift green to red
#print('[original distance, corrected distance (shift red to green), corrected distance (shift green to red)]' )
#for i in np.arange(len(positions_green)):
#     g0=positions_green[i]
#     g=np.dot(my_homography_inverse,[positions_green[i,0],positions_green[i,1],1])[0:2]  # Correct green/blue
#     r0=positions_red[i]
#     r=np.dot(my_homography,[positions_red[i,0],positions_red[i,1],1])[0:2]  # What you'll do: shift red to green
#     print([np.linalg.norm(g0-r0),np.linalg.norm(g0-r),np.linalg.norm(g-r0)]) # Check corrections

In [3]:
# Read in the original 3D video 
video_3D = imread(video_3D_filename_path+video_3D_filename)

In [6]:
# Read in the tracking file that corresponds to the original 3D video
tracks = pd.read_csv(video_3D_filename_path+track_filename) 
# tracks.head()

In [8]:
viewer = napari.Viewer()
viewer.add_image(video_3D[:,:,:,:,0],name='red',colormap = 'red',blending="additive", scale=[z_renorm,1,1])
viewer.add_image(video_3D[:,:,:,:,1],name='green',colormap = 'green',blending="additive", scale=[z_renorm,1,1])
viewer.add_image(video_3D[:,:,:,:,2],name='blue',colormap = 'blue',blending="additive", scale=[z_renorm,1,1])
viewer.add_tracks(tracks[['TRACK_ID','POSITION_T','POSITION_Z','POSITION_Y','POSITION_X']].values, name="TRACK_IDs")

<Tracks layer 'TRACK_IDs' at 0x2005a7a3d88>

In [ ]:
create_track_array_video(working_directory, track_array_filename, video_3D, 
                             tracks, crop_pad, xy_pixel_size, z_pixel_size)#, homography = my_homography)

### Track Array bg-correction one-by-one: best-z projection + 2D disk/donut

In [9]:
# Just in case you want to update trackArrayTools without having to restart Kernel
#del sys.modules['trackArrayTools']  # Use this if you are coding new class methods and want to reread trackArrayTools.py file
#from trackArrayTools import *

# Choose which track array you want to work with:
working_directory ='C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Our papers/Ago2Tethering/TimFastTrackAnalysis/'
track_array_filename = 'Tracks_TA02_90m_crop_pad_5.tif'

# The corresponding tracking file should be in the same directory and have the following filename: 
track_filename = track_array_filename[:-15] + '.csv'

# Read in the track array video 
track_array_vid = imread(working_directory + track_array_filename)  # Read in track array .tif file

# Read in tracks as a "dataframes (df)"
track_array_df = pd.read_csv(working_directory + track_filename)

# Make sure color channels are the last dimension of the track array
dims = list(track_array_vid.shape)
if len(dims) != 3:     # check if just a single channel video
    n_channels = min(dims)
    n_channels_index = dims.index(n_channels)
    track_array_vid = np.moveaxis(track_array_vid,n_channels_index,-1)

In [11]:
# Step 0: Create the track array object:
ta = TrackArray(track_array_vid, track_array_df, crop_pad)

In [12]:
# Step 0.5: check if z's are offset in different channels
my_offset = [0,0,0]  ### For 3-color, could be [0,-1,-1]; BE CAREFUL, THIS WILL MAKE A MASK THAT IS SHIFTED IN Z FOR BLUE AND GREEN!!!
capsule_mask = ta.capsule_mask(crop_pad,crop_pad,2,1,1,z_offset=my_offset)
arr_bg = ta.local_background_subtract(ta.arr,capsule_mask)

# Check with napari
max = np.max(arr_bg,axis=(0,1,2)) # find max intensities for setting intensity range
my_range = [[0,max[ch]] for ch in np.arange(len(max))]
napari_viewer(arr_bg,[z_renorm,1,1],int_range=my_range,layer=[capsule_mask[:,:,:,0],capsule_mask[:,:,:,1]])

In [13]:
# Step 1: Best-z projection using offset from above
my_best_z_masks = ta.best_z_mask(1,1,z_offset = my_offset) # best-z mask w/ offset
best_z = ta.mask_projection(np.clip(ta.arr,0,1000000), my_best_z_masks) # best-z projection; clip to make sure data is +

# Check with napari
max = np.max(best_z,axis=(0,1)) # find max intensities for setting intensity range
my_range = [[0,max[ch]] for ch in np.arange(len(max))]
napari_viewer(best_z,[1,1],int_range=my_range)

In [14]:
# Step 2: Subtract background in donut rings from max projection  
best_z_bg = ta.local_background_subtract(best_z,ta.donut_mask_2D(5,1))

# Check with napari and show donut mask
max = np.max(best_z_bg,axis=(0,1)) # find max intensities for setting intensity range
my_range = [[0,max[ch]] for ch in np.arange(len(max))]
napari_viewer(best_z_bg,[1,1],int_range=my_range,layer=[ta.donut_mask_2D(5,1)])

In [ ]:
# Output background-subtracted and max-z projected track array 
output_filename_path = working_directory + track_array_filename[0:-4] + '_bestz_bg-sub.tif'
io.imsave(output_filename_path,
        best_z_bg, 
        resolution=(1/xy_pixel_size,1/xy_pixel_size))
output_filename_path